In [1]:
import torch

# Código que utiliza la GPU

# Liberar la memoria de la GPU
torch.cuda.empty_cache()

In [2]:
CUDA_LAUNCH_BLOCKING=1

In [31]:
!pip install langchain
!pip install logchain
!pip install langchain-community
!pip install pypdf
!pip install sentence-transformers
!pip install safetensors


In [18]:
import logchain
from langchain.document_loaders import PyPDFLoader

# Load the PDF file from current working directory
loader = PyPDFLoader("pdfexample.pdf")

# Split the PDF into Pages
pages = loader.load_and_split()

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define chunk size, overlap and separators
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=64,
    separators=['\n\n', '\n', '(?=>\. )', ' ', '']
)

# Split the pages into texts as defined above
texts = text_splitter.split_documents(pages)

In [20]:
# Print the number of pages and number of texts CUDA Setup failed despite GPU being available. Please run the following command to get more information:
print(len(pages), len(texts))
# 27, 72

38 92


In [21]:
# Print a full page of text
print(pages[2].page_content)

# Print the split text
#print(texts[50].page_content)

# Print the len of the above split text. Note it will not be more than 1024.
#print(len(texts[50].page_content))
# 1023

3 protección jurídica destinada a su reconocimiento y pervivencia que el ordenamiento jurídico colombiano no contempla en la actualidad. Por este motivo se propone la inclusión en el informe final de una referencia al valor de este tipo de iniciativas, así como la recomendación de propugnar la emisión de una Ley de Memoria Histórica que incluya un procedimiento adecuado de reconocimiento, identificación y protección de los espacios de memoria generados por la interacción de víctimas y sociedad civil en los territorios.    Índice.  Información básica _______________________________________________ 4 a. Tema del informe ________________________________________ 4 b. Delimitación espacial y temporal del informe __________________ 5 c. Identificación del grupo que presenta el informe ________________ 5 d. Relevancia del informe ____________________________________ 5  Metodología ____________________________________________________ 6 Hallazgos y análisis _____________________________________

In [24]:
from langchain.embeddings import HuggingFaceEmbeddings

# Load embeddings from HuggingFace
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [25]:
from langchain.vectorstores import SKLearnVectorStore
# Set the persisted vector store
vector_db_path = "./document_vector_db.parquet"

# Create the vector store
vector_db = SKLearnVectorStore.from_documents(texts, embedding=embedding, persist_path=vector_db_path,
                                             serializer="parquet")

In [26]:
print(vector_db.similarity_search("protección jurídica"))

[Document(metadata={'id': 'bf734245-6831-429a-aa2e-2ed4e1978322', 'source': 'pdfexample.pdf', 'page': 5}, page_content='6 consecución el Acto Legislativo 01 de 2017 incorporó a la Constitución Política los tres mecanismos del Sistema Integral de Verdad, Justicia, Reparación y No Repetición). Asimismo, advierte acerca del riesgo que para la existencia futura de los espacios de memoria generados por la interacción de víctimas y colectivos de acción social supone la falta en el ordenamiento jurídico colombiano de instrumentos normativos con capacidad para otorgarles una protección legal adecuada. Este documento se articula en torno a la convicción de que los sitios de memoria desempeñan un papel esencial en la garantía del derecho a la verdad del conjunto de la sociedad, por la relevancia que tiene la participación de las víctimas, sus familias y la comunidad local en la configuración de una memoria colectiva que entre en diálogo con la oficial patrocinada por el Estado y sirva de dique d

In [27]:
import pandas as pd

# persist the store
vector_db.persist()

# load into pandas
df = pd.read_parquet(vector_db_path)

# Have a look at the store
df

# Show first row of store
# df.iloc[0]
#ids                        94fb087f-6206-430d-8fb3-2549b1c64d1a
#texts         A Beginner’s Guide to \nLarge Language Models\...
#metadatas          {'page': 0, 'source': 'llm-ebook-part1.pdf'}
#embeddings    [0.033385541290044785, 0.020513998344540596, -...
#Name: 0, dtype: object

,ids,texts,metadatas,embeddings
0,db8a8123-6ae3-40ea-95d0-ecadf9ed2dfd,1,"{'page': 0, 'source': 'pdfexample.pdf'}","[0.012244479730725288, -0.009157432243227959, ..."
1,0e0e36ca-dbba-45af-a8db-b18923b5815c,2 Señores grupo de gestión documental Comisió...,"{'page': 1, 'source': 'pdfexample.pdf'}","[0.006308652460575104, 0.08947838842868805, 0...."
2,7eefbf52-c61d-4973-a675-c8285ebe1f1f,“promover el reconocimiento de los procesos de...,"{'page': 1, 'source': 'pdfexample.pdf'}","[-0.015396995469927788, 0.08079889416694641, 0..."
3,6f613eb9-9e78-4d9f-86cb-3c0532f840a3,3 protección jurídica destinada a su reconocim...,"{'page': 2, 'source': 'pdfexample.pdf'}","[0.01033114269375801, 0.06237919628620148, -0...."
4,0c4e52a6-da74-46dd-a792-77c7dce2ad6b,______________________________________________...,"{'page': 2, 'source': 'pdfexample.pdf'}","[-0.01565587893128395, 0.05366784706711769, 0...."
...,...,...,...,...
87,25af3233-32d5-40c4-a632-c8228553856e,"35 Para el lanzamiento de la exposición, la Ga...","{'page': 34, 'source': 'pdfexample.pdf'}","[-0.0011809037532657385, 0.04692536219954491, ..."
88,e2280acc-da26-450d-939c-f9cf057d4b04,"de artistas en el cementerio, para la creación...","{'page': 34, 'source': 'pdfexample.pdf'}","[-0.032969050109386444, 0.1045699492096901, -0..."
89,c69b79b1-2291-4f74-8ac8-1b69789cd478,36 Ejecución de la exposición en un recorrido...,"{'page': 35, 'source': 'pdfexample.pdf'}","[-0.001731930417008698, 0.07147882878780365, -..."
90,89f4bec1-adfd-4981-94d0-418d5ca4fa41,37 Homenaje a las doñas del barrio Nuevos Conq...,"{'page': 36, 'source': 'pdfexample.pdf'}","[-0.0018223010702058673, 0.06712042540311813, ..."


In [28]:
import re
import warnings
from typing import List

import torch
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    pipeline,
)

warnings.filterwarnings("ignore", category=UserWarning)

In [33]:
from transformers import AutoModelForQuestionAnswering

model_name = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    #load_in_8bit=True,
    device_map='auto',
    offload_folder="content/offload_files"
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
generate_text = pipeline(task="text-generation", model=model, tokenizer=tokenizer,
                         trust_remote_code=True, max_new_tokens=100,
                         repetition_penalty=1.1, model_kwargs={"device_map": "auto",
                          "max_length": 200, "temperature": 0.01, "torch_dtype":torch.bfloat16}
)
# LangChain HuggingFacePipeline set to our transformer pipeline
hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=hf_pipeline, chain_type="stuff",
                                 retriever=vector_db.as_retriever(search_kwargs={"k": 3}),
                                 return_source_documents=True,
                                 verbose=False,
)


In [ ]:
query1 = "¿Quién fue el director del programa de TEDESOFT en el 2020"
query = "¿Qué significa la EISC en la Univalle?"
# Send question as a query to qa chain
result = qa({"query": query})

/home/raul/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [ ]:
print(result)

{'query': '¿Qué significa la EISC en la Univalle?', 'result': '\n\nThe EISC (Ejecucion de Instrumentos de Sonorizacion y Canto) is a musical instrument that is used in the performing arts. It is a type of electronic keyboard that is designed to mimic the sound of a piano, but with more advanced features. The EISC is commonly used in musical productions and performances, as it allows for a wide range of sounds and tones to be produced.', 'source_documents': [Document(page_content='ejecución de partituras.', metadata={'id': 'e62c0782-0399-4259-b45a-fd0c6e2de625', 'page': 648, 'source': 'arcusur.pdf'}), Document(page_content='ejecución de partituras.', metadata={'id': '1ec74fdd-e42e-4212-a8fa-d1d8c259df35', 'page': 648, 'source': 'arcusur.pdf'}), Document(page_content='ejecución de partituras.', metadata={'id': '7806108f-b0ff-4006-88d1-1888e31a1c98', 'page': 648, 'source': 'arcusur.pdf'})]}


In [ ]:
print(result["result"])
# \nThis book is for anyone interested in learning about large language models, including researchers, developers, and students. It covers the basics of language models, their applications, and their potential benefits for businesses and individuals.



The EISC (Ejecucion de Instrumentos de Sonorizacion y Canto) is a musical instrument that is used in the performing arts. It is a type of electronic keyboard that is designed to mimic the sound of a piano, but with more advanced features. The EISC is commonly used in musical productions and performances, as it allows for a wide range of sounds and tones to be produced.


In [ ]:
print(len(result['source_documents']))
# 3

3


In [ ]:
# Print page document source and page number of first source_document
print(result['source_documents'][0].metadata['source'], result['source_documents'][0].metadata['page'])
# llm-ebook-part1.pdf 7

arcusur.pdf 648


In [ ]:
from datasets import disable_caching
disable_caching()

In [ ]:
import torch

class ClearCache:
    def __enter__(self):
        torch.cuda.empty_cache()

    def __exit__(self, exc_type, exc_val, exc_tb):
        torch.cuda.empty_cache()

# Use the context manager
with ClearCache():
    # Define and train the PyTorch model
    ...